In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(1)

In [2]:
word_to_ix = {'hello' : 0, 'world' : 1}
embedds = nn.Embedding(2, 5)
lookup_tensor = torch.tensor([word_to_ix['hello']], dtype = torch.long)
hello_embedd = embedds(lookup_tensor)
print("Printing Word Embedding for \"hello\" : {}".format(hello_embedd))

Printing Word Embedding for "hello" : tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]])


In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence ="""When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i+1]],
             test_sentence[i+2]) 
            for i in range(len(test_sentence) - 2)]

print("First three trigrams :\n{}".format(trigrams[:3]))
print("-" * 80)

vocab = set(test_sentence)
word_to_ix = {word : i for i, word in enumerate(vocab)}

for key, val in word_to_ix.items():
    print("{} : {}".format(key, val))
print("-" * 80)

First three trigrams :
[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
--------------------------------------------------------------------------------
a : 0
thy : 1
be : 2
new : 3
all : 4
in : 5
thou : 6
it : 7
field, : 8
own : 9
When : 10
treasure : 11
Thy : 12
were : 13
where : 14
to : 15
say, : 16
'This : 17
praise. : 18
cold. : 19
answer : 20
worth : 21
deep : 22
by : 23
mine : 24
forty : 25
Then : 26
count, : 27
child : 28
shall : 29
and : 30
of : 31
days; : 32
thine : 33
Will : 34
being : 35
within : 36
deserv'd : 37
weed : 38
his : 39
To : 40
How : 41
thriftless : 42
use, : 43
eyes, : 44
sunken : 45
beauty's : 46
youth's : 47
This : 48
asked, : 49
the : 50
thine! : 51
made : 52
praise : 53
And : 54
trenches : 55
feel'st : 56
proud : 57
excuse,' : 58
all-eating : 59
beauty : 60
sum : 61
totter'd : 62
gazed : 63
dig : 64
art : 65
Shall : 66
warm : 67
much : 68
so : 69
If : 70
held: : 71
more : 72
see : 73
brow, : 74
now, : 75
l

In [4]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedd_size, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedd_size)
        self.lin1 = nn.Linear(context_size * embedd_size, 128)
        self.lin2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embedds = self.embeddings(inputs).view(1, -1)
        out = F.relu(self.lin1(embedds))
        out = self.lin2(out)
        log_probab = F.log_softmax(out, dim = 1)
        
        return log_probab

In [5]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(vocab_size = len(vocab), embedd_size = EMBEDDING_DIM, context_size = CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[521.2529845237732, 518.6605763435364, 516.0867598056793, 513.5304462909698, 510.9892761707306, 508.463095664978, 505.9510407447815, 503.4524276256561, 500.9663953781128, 498.4916613101959]


In [6]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers. As they
evolve, processes manipulate other abstract things called data. The evolution
of a process is directed by a pattern of rules called a program. People create
programs to direct processes. In effect, we conjure the spirits of the computer
with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word : i for i, word in enumerate(vocab)}
ix_to_word = {i : word for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [7]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedd_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedd_size)
        self.lin1 = nn.Linear(embedd_size, 128)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(128, vocab_size)
        self.activation = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embedd = sum(self.embedding(inputs)).view(1, -1)
        out = self.lin1(embedd)
        out = self.relu(out)
        out = self.lin2(out)
        activation = self.activation(out)
        
        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    print(context)
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [8]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = CBOW(vocab_size, EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

for epoch in range(300):
    total_loss = 0
    for context, target in data:
        
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    if epoch % 10 == 0:
        print('Loss in epoch : {} is {}'.format(epoch, total_loss))

Loss in epoch : 0 is 236.44623708724976
Loss in epoch : 10 is 194.8331024646759
Loss in epoch : 20 is 167.59508979320526
Loss in epoch : 30 is 144.87409341335297
Loss in epoch : 40 is 125.04833954572678
Loss in epoch : 50 is 107.64246815443039
Loss in epoch : 60 is 92.39136058092117
Loss in epoch : 70 is 79.1507219672203
Loss in epoch : 80 is 67.68786174058914
Loss in epoch : 90 is 57.84135642647743
Loss in epoch : 100 is 49.4972825050354
Loss in epoch : 110 is 42.47349938750267
Loss in epoch : 120 is 36.60188786685467
Loss in epoch : 130 is 31.70926709473133
Loss in epoch : 140 is 27.64286206662655
Loss in epoch : 150 is 24.25125727057457
Loss in epoch : 160 is 21.41305112838745
Loss in epoch : 170 is 19.027796544134617
Loss in epoch : 180 is 17.01114085316658
Loss in epoch : 190 is 15.297546200454235
Loss in epoch : 200 is 13.83368019014597
Loss in epoch : 210 is 12.575558044016361
Loss in epoch : 220 is 11.488856919109821
Loss in epoch : 230 is 10.544785313308239
Loss in epoch : 240

In [9]:
def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

In [10]:
context = ['People','create', 'to', 'direct']
context_vector = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Context: {}\n'.format(context))
print('Prediction: {}'.format(get_max_prob_result(a[0], ix_to_word)))


Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs


**JOB DONE!**